In [ ]:
from pyomo.environ import *
from pyomo.gdp import *

In [ ]:
CLEANING_TIMES = [
    [0, 11, 7, 13, 11],
    [5, 0, 13, 15, 15],
    [13, 15, 0, 23, 11],
    [9, 13, 5, 0, 3],
    [3, 7, 7, 7, 0]
]

MIXING_TIMES = [40, 35, 45, 32, 50]